In [1]:
import matplotlib.pyplot as plt
import cv2
import PIL.Image
import os
import time
import re
from google.api_core.exceptions import ResourceExhausted
import json
from utils import *

/Users/saptarshimallikthakur/Desktop/google_lens/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
output_dict = {}

image_dir = 'datasets/fashion_dataset/train/images'

# Your list of filenames
image_list =sorted([f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))])
numbers = sorted([int(re.search(r'image_(\d+)', filename).group(1)) for filename in image_list])

start = numbers[0]

while True:
    try:
        for i in numbers[numbers.index(start):]:

            image_path = f'datasets/fashion_dataset/train/images/image_{i}.jpg'

            # print("Processing:", image_path)
            detection = detect_clothing(image_path, model_yolo, labels, upper, lower)

            if len(detection) > 0:
                output_dict[image_path] = {}

            for j, detect in enumerate(detection):
                try:
                    response_text = process_image(detect['cropped_image'])
                    category = detect['category']
                    output_dict[image_path][category] = response_text
                except ResourceExhausted as e:
                    print(f"ResourceExhausted encountered: {e}. Waiting 20s and restarting.")
                    start = i  # Restart from the current image
                    time.sleep(20)
                    raise  # Reraise to restart the outer loop
                except Exception as e:
                    print(f"Error processing detection in image {image_path}: {e}")
                    print("Skipping this detection.")
                    continue  # Skip to the next detection

    except Exception as e:
        print(f"Error encountered during processing: {e}")
        print("Restarting from the last processed image.")
        continue  # Restart the while loop
    else:
        print("Processing complete!")
        break

In [ ]:
# Save the dictionary to a file
with open('Cloth Description.json', 'w') as file:
    json.dump(output_dict, file)